In [ ]:
import numpy as np
import numba
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp

# $\frac{d^2\theta}{dt^2} = -\omega_0^2\sin\theta -\alpha\frac{d\theta}{dt} + f\cos\omega t$

In [ ]:
from typing import Callable


def make_pendulum_ode_func(
    omega_0: float, alpha: float, f: float, omega: float
) -> Callable[[float, np.ndarray], np.ndarray]:
    def ode_func(t: float, x: np.ndarray) -> np.ndarray:
        theta, dtheta = x

        d2theta = (
            -(omega_0**2) * np.sin(theta) - alpha * dtheta + f * np.cos(omega * t)
        )

        return np.array([dtheta, d2theta])

    return ode_func

In [ ]:
def solve_pendulum(
    t: np.ndarray,
    y0: np.ndarray,
    omega_0: float,
    alpha: float,
    f: float,
    omega: float = np.pi,
) -> tuple[np.ndarray, np.ndarray]:
    """[theta, dtheta]"""
    pendulum = make_pendulum_ode_func(omega_0, alpha, f, omega)

    t_min = 0
    t_max = np.max(t)

    sol = solve_ivp(pendulum, [t_min, t_max], y0, t_eval=t)

    theta = sol.y[0, :]
    dtheta = sol.y[1, :]

    return theta, dtheta

In [ ]:
t = np.linspace(0, 10, 100)
theta_0 = 0.1
dtheta_0 = 0.05
y0 = np.array([theta_0, dtheta_0])
theta, dtheta = solve_pendulum(t, y0, 1, 0, 0)

In [ ]:
plt.figure()

plt.subplot(1, 2, 1)
plt.plot(t, theta)
plt.xlabel("t")
plt.ylabel("$\\theta$")
plt.title("Time evolution")

plt.subplot(1, 2, 2)
plt.plot(theta, dtheta)
plt.xlabel("$\\theta$")
plt.ylabel("$d\\theta$")
plt.title("Phase space")

plt.tight_layout()
plt.show()

In [ ]:
def plot_with_random_y0(omega_0, alpha, f, omega = np.pi):
    plt.figure()
    t = np.linspace(0, 10, 40)
    for _ in range(30):
        theta_0 = np.random.uniform(-2*np.pi, 2*np.pi)
        dtheta_0 = np.random.uniform(-np.pi, np.pi)
        y0 = np.array([theta_0, dtheta_0])
        theta, dtheta = solve_pendulum(t, y0, omega_0, alpha, f)
        plt.scatter(theta, dtheta, c="tab:blue")
    plt.title(f"{omega_0 = }, {alpha = }, {f = }")
    plt.show()

In [ ]:
plot_with_random_y0(1, 0, 0)

In [ ]:
plot_with_random_y0(1, 0.1, 0)
# Ooh, spirals!

In [ ]:
plot_with_random_y0(1, 0.1, 0.5, 2)

In [ ]:
# overwrite the old one with the sin-dependent perturbation
def make_pendulum_ode_func(
    omega_0: float, alpha: float, f: float, omega: float
) -> Callable[[float, np.ndarray], np.ndarray]:
    def ode_func(t: float, x: np.ndarray) -> np.ndarray:
        theta, dtheta = x

        d2theta = (
            (- omega_0**2 + f * np.cos(omega * t)) * np.sin(theta) - alpha * dtheta 
        )

        return np.array([dtheta, d2theta])

    return ode_func

In [ ]:
t = np.linspace(100, 110, 100)
t[0] = 0

theta_0 = np.random.uniform(-2*np.pi, 2*np.pi)
dtheta_0 = np.random.uniform(-np.pi, np.pi)
y0 = np.array([theta_0, dtheta_0])

fs = np.linspace(0, 2, 20)

dtheta_at_theta_0 = []

for f in fs:
    theta, dtheta = solve_pendulum(t, y0, 1, 0, 0)
